<a href="https://colab.research.google.com/github/Gaurav1401/Zenatix_Solutions/blob/main/Part%20-%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# pip install neuralprophet

In [58]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from neuralprophet import NeuralProphet
from sklearn.preprocessing import MinMaxScaler
from multiprocessing import Pool, cpu_count

In [2]:
final = pd.read_csv("https://raw.githubusercontent.com/Gaurav1401/Zenatix_Solutions/main/another_data.csv")

In [3]:
final.head()

,index,Timestamp,AC 1,AC 2,AC 3,AC 4,AC 5,AC 6,AC 7,AC 8,AC 9,AC 10,AC 11,AC 12,AC 13,AC 14,AC 15,AC 16,AC 17,AC 18
0,0.0,2019-08-01 00:00:00,7.518632,8.788315,0.000000,0.000000,2.617045,4.079041,2.782276,4.624447,5.222060,2.151238,1.585072,0.560373,3.142941,2.749470,5.417774,4.113460,3.305072,6.735981
1,1.0,2019-08-01 00:01:00,7.341986,8.869652,0.207528,-0.099235,2.593492,4.070208,2.765041,4.675383,5.170101,2.285380,1.553008,1.004822,3.356319,2.827610,5.451596,3.925502,3.399600,6.925839
2,2.0,2019-08-01 00:02:00,7.426114,8.940615,0.000000,0.000000,2.581625,3.781231,2.529366,5.057423,5.349465,2.414715,2.168184,1.818730,3.085110,2.720484,3.302422,3.986483,3.220588,6.379500
3,3.0,2019-08-01 00:03:00,7.141880,8.807515,0.441401,-0.079926,2.560526,4.065357,2.622586,4.715786,4.910319,2.522343,1.513539,1.591031,3.681013,3.058494,5.547122,3.728646,3.278150,7.050406
4,4.0,2019-08-01 00:04:00,7.052986,9.161103,0.000000,0.000000,2.592095,3.800127,2.332304,6.322521,3.995392,2.237114,3.345624,2.310409,3.132799,2.676861,3.539026,3.797881,3.131560,6.363475


In [4]:
timestamp = final["Timestamp"]
final.drop("Timestamp", axis = "columns", inplace = True)

In [5]:
final[final < 0] = 0
final[final > 11.2] = 11.2

In [6]:
final.insert(0, "Timestamp", timestamp)

In [7]:
final.drop("index", axis = "columns", inplace = True)

In [8]:
final = final.set_index("Timestamp")

In [9]:
final.iloc[37440:-120, :].shape

(50280, 18)

**I am including the data of only last 5 days of the month of August as for the predictions of November month, the valeus won't be much dependent on the values of August Month**

In [65]:
train = final.iloc[37440:-120, :] 
test = final.iloc[-120:, :]

In [26]:
data = train['AC 1'].reset_index()
data.columns = ['ds', 'y']

In [33]:
model = NeuralProphet() # won't include confidence intervals and will also save time
model.fit(data,freq = "min")
future = model.make_future_dataframe(data, periods=120, n_historic_predictions=len(train)) #won't include training data in the output
forecast = model.predict(future)

INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 68


  0%|          | 0/335 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.21E-02, min: 7.43E-01


  0%|          | 0/335 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 7.49E-02, min: 8.42E-01


  0%|          | 0/335 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.48E-02, min: 7.91E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 7.33E-02
Epoch[68/68]: 100%|██████████| 68/68 [01:33<00:00,  1.38s/it, SmoothL1Loss=0.0321, MAE=1.56, RMSE=2.07, RegLoss=0]


In [39]:
forecast.tail()

,ds,y,yhat1,residual1,trend,season_weekly,season_daily
50395,2019-09-30 23:55:00,NaN,1.744359,NaN,1.127576,0.872516,-0.255734
50396,2019-09-30 23:56:00,NaN,1.743328,NaN,1.127307,0.873007,-0.256986
50397,2019-09-30 23:57:00,NaN,1.742334,NaN,1.127035,0.873497,-0.258199
50398,2019-09-30 23:58:00,NaN,1.741380,NaN,1.126764,0.873987,-0.259371
50399,2019-09-30 23:59:00,NaN,1.740466,NaN,1.126493,0.874477,-0.260504


In [37]:
forecast.iloc[-120:, 2]

50280    1.975827
50281    1.974182
50282    1.972513
50283    1.970822
50284    1.969105
           ...   
50395    1.744359
50396    1.743328
50397    1.742334
50398    1.741380
50399    1.740466
Name: yhat1, Length: 120, dtype: float64

In [66]:
scaler = MinMaxScaler()
cols = train.columns
train_index = train.index
test_index = test.index
train = pd.DataFrame(scaler.fit_transform(train), columns = cols,index = train_index)
test = pd.DataFrame(scaler.transform(test), columns = cols, index = test_index)

In [67]:
test.head()

,AC 1,AC 2,AC 3,AC 4,AC 5,AC 6,AC 7,AC 8,AC 9,AC 10,AC 11,AC 12,AC 13,AC 14,AC 15,AC 16,AC 17,AC 18
Timestamp,,,,,,,,,,,,,,,,,,
2019-09-30 22:00:00,0.217697,0.541550,0.130331,0.171925,0.058221,0.018166,0.151380,0.215550,0.183654,0.108070,0.135068,0.178230,0.370389,0.235679,0.259873,0.076365,0.412040,0.653190
2019-09-30 22:01:00,0.216960,0.545823,0.133271,0.174000,0.056481,0.015610,0.157589,0.214117,0.180849,0.109827,0.140222,0.179066,0.379205,0.236498,0.259529,0.075026,0.411312,0.653227
2019-09-30 22:02:00,0.217397,0.548588,0.136451,0.174472,0.054091,0.013630,0.163820,0.212202,0.178439,0.111831,0.145118,0.179566,0.388179,0.237511,0.259319,0.073917,0.410617,0.653337
2019-09-30 22:03:00,0.217560,0.553756,0.139570,0.175792,0.052052,0.011109,0.170636,0.211042,0.176017,0.113647,0.150751,0.180223,0.396266,0.237796,0.259019,0.074160,0.409887,0.653115
2019-09-30 22:04:00,0.218183,0.557186,0.142379,0.176440,0.049399,0.009020,0.176709,0.208244,0.172178,0.116215,0.155454,0.180447,0.403695,0.238170,0.258712,0.074480,0.409247,0.652974


In [68]:
def run_prophet(timeseries):
    model = NeuralProphet() 
    model.fit(timeseries,freq = "min",epochs = 25)
    future = model.make_future_dataframe(timeseries, periods=120, n_historic_predictions=len(timeseries)) 
    forecast = model.predict(future)
    return forecast

In [69]:
predictions = pd.DataFrame()

In [70]:
for i in test.columns:
  train1 = pd.DataFrame()
  train1 = train[[i]].reset_index()
  train1.columns = ['ds', 'y']



  forecast = run_prophet(train1)


  predictions[i] = forecast.iloc[-120:, 2]

INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64


  0%|          | 0/335 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.28E-02, min: 1.77E+00


  0%|          | 0/335 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.03E-02, min: 1.57E+00


...

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.14E-02, min: 1.30E+00


  0%|          | 0/335 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.78E-02, min: 9.53E-01


  0%|          | 0/335 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.78E-02, min: 8.96E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 3.56E-02
Epoch[25/25]: 100%|██████████| 25/25 [00:34<00:00,  1.39s/it, SmoothL1Loss=0.0287, MAE=0.152, RMSE=0.188, RegLoss=0]


In [74]:
predictions_final = pd.DataFrame(scaler.inverse_transform(predictions), columns = cols, index = test_index)

# **Evaluation**

In [75]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [76]:
# Evaluating using Mean Absolute Percentage Error
lst = []
for i in test.columns:
    error = mape(test[i], predictions_final[i])
    print(f"{i} ----> {error}")
    lst.append(error)

WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until



AC 1 ----> 845.890289978092
AC 2 ----> 270.09144558625377
AC 3 ----> 755.0314808775013
AC 4 ----> inf
AC 5 ----> 2962.173141398896
AC 6 ----> inf
AC 7 ----> 667.9186521634289
AC 8 ----> inf
AC 9 ----> 1399.4984776138149
AC 10 ----> 920.3682586096056
AC 11 ----> 1123.9925493193991
AC 12 ----> 445.4771987032642
AC 13 ----> 345.4011371918405
AC 14 ----> 1691.6373856442513
AC 15 ----> 492.1050610438955
AC 16 ----> 1677.1332658662122
AC 17 ----> 1049.3993313857382
AC 18 ----> 672.7880566999833
